# Setup

In [1]:
%pip install openai tiktoken

Note: you may need to restart the kernel to use updated packages.


In [61]:
from openai import OpenAI
client = OpenAI()
llm = "gpt-4-1106-preview"

In [129]:
job_options=['medical-assistant', 'infra-engineer', 'customer-success-manager', 'real-estate-litigator','growth-manager','product-designer']

choice = job_options[5]

job_description_file = "examples/" + choice + "/job-description.md"
company_file = "examples/" + choice + "/about-company.md"

# destination files
destination_directory = "examples/" + choice + "/"
draft_destination = destination_directory + choice + '-draft.md'
markdown_destination = destination_directory + choice + '-playbook.md'
word_destination = destination_directory + choice + '-playbook.docx'



In [130]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        # print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        # print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens


def num_tokens_from_string(string, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a string."""
    return num_tokens_from_messages([{"content": string}], model=model)


# Load the Assistant

In [137]:
from datetime import datetime


# basic instructions
system_prompt = """
You are a Talent Acquisition and Development Specialist. 
Your task is to do a job analysis and identify the knowledge, skills, abilities, and other characteristics (KSAOs) as 5-10 competencies for a hiring team to assess in a candidate. 
At least 4 competencies should cover the technical skills of the job.
Lower level jobs should have less competencies and higher level jobs should have more, but you should cover all the requirements in the job desccription. 
You should also include competencies or word competencies that are aligned with the company values and if a competency is aligned with a company value, you should say so.
Cite and quote the job description as appropriate.
Include the job description verbatim, but change the heading levels to be just bolded text.
Be sure to cover specific technologies, systems or skills mentioned in the job description in the competencies or competency detail.

Tone: spartan, use less corporate jargon. Write like Hemingway.
Don't use the word 'technological', 'tranformational', 'foster', 'fostering', 'tapestry', 'like', 'overall', 'furthermore'
Don't use the phrases 'this is about', 'all about', 'think of', 'not only', 'but also', 'in conclusion'
Refer to the actual company name rather than 'organization' or 'company'.
Vary your sentence length and structure.
Avoid repeating the same word in a sentence or paragraph.

If you are asked to review the document, review the quality and accuracy of the competencies and the interview questions.
Do NOT modify the job description as this has been provided to you by the hiring manager.
Cite and or quote the job description as appropriate.
Be sure to cover specific technologies, systems or skills mentioned in the job description in the competencies or competency detail.

Make sure that you always follow the template from your system instructions EXACTLY and completely.

"""

# system_prompt += "\n\nFor this test, only produce 1 competencies."

# add today's date and time to include in the document
system_prompt += "\nToday's date and time is " + datetime.now().strftime("%B %d, %Y %-I:%M %p") + ".\n"

# instructions for processing the document template. We will reuse this.
doc_prompt = """
You will be provided below with a template in Jinja2 syntax delimited by triple quotes, and your task is to fully and accurately populate the template.
Always complete all loops fully. 
Always complete the metadata.
Always include a blank line after a markdown block and before a list.
Do NOT ommit or truncate any content or sections. 
Do NOT ommit blank spaces or lines.
Do NOT include any delimiters or commentary in the output.
Where prompted, provide your thinking about what you will write and include your thinking in HTML comment tags.

When you are completely finished with the entire document write STOPSTOP.

The Jinja2 template that you will follow faithfully is as follows:
"""
system_prompt += doc_prompt

# add the document template
with open('templates/prompt_template.md', 'r') as file:
    document_template = file.read()

system_prompt += "\n\"\"\"\n" + document_template + "\n\"\"\"\n\n"
    
print(system_prompt)


You are a Talent Acquisition and Development Specialist. 
Your task is to do a job analysis and identify the knowledge, skills, abilities, and other characteristics (KSAOs) as 5-10 competencies for a hiring team to assess in a candidate. 
At least 4 competencies should cover the technical skills of the job.
Lower level jobs should have less competencies and higher level jobs should have more, but you should cover all the requirements in the job desccription. 
You should also include competencies or word competencies that are aligned with the company values and if a competency is aligned with a company value, you should say so.
Cite and quote the job description as appropriate.
Include the job description verbatim, but change the heading levels to be just bolded text.
Be sure to cover specific technologies, systems or skills mentioned in the job description in the competencies or competency detail.

Tone: spartan, use less corporate jargon. Write like Hemingway.
Don't use the word 'tec

## Job Description

In [138]:
def get_job_description():
    job_description = ""
    with open(job_description_file, "r") as file:
        job_description = file.read()
    return job_description

def get_company_description():
    company_description = ""
    with open(company_file, "r") as file:
        company_description = file.read()
    return company_description

def build_user_prompt(job_description, company_description):
    user_prompt = "I am supplying you with a job description and a company description.\n\n"
    user_prompt += "Job description is as follows:\n\n\"\"\"\n" + job_description + "\n\"\"\"\n\n"
    user_prompt += "Company description is as follows:\n\n\"\"\"\n" + company_description + "\n\"\"\"\n\n"
    user_prompt += "Proceed generating the full document. Do not truncate or abridge the document. You MUST follow the template EXACTLY.\n\n"
    return user_prompt

job_description = get_job_description()
company_description = get_company_description()

user_prompt = build_user_prompt(job_description, company_description)

print(user_prompt)


I am supplying you with a job description and a company description.

Job description is as follows:

"""
## Job Title: Product Designer

### About Tiny

If you've ever written content on the web, chances are you've used a Tiny product. As part of the Tiugo Technologies portfolio of brands, we make TinyMCE and other great developer tools. Our team is dedicated to bringing content creation tools globally through open-source initiatives.

We're expanding our product family and are looking for a talented Product Designer to join our remote team of over 200 members worldwide, with significant presence in Australia, Poland, and the USA.

### About the Role

As a Product Designer for TinyMCE, you will have a direct impact on millions of users globally. In the TinyMCE product team, you will be part of the entire development journey, from identifying issues to defining and designing solutions. Your responsibilities include creating wireframes, prototypes, UI designs, and considering the broade

## Product the Draft

In [139]:
import os

def generate_output(system_prompt, user_prompt, llm):
    STOP_SIGNAL = "STOPSTOP"
    output = "" 

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    input_tokens = 0
    output_tokens = 0

    while True:
        try:
            stream = client.chat.completions.create(
                model=llm,
                max_tokens=4096,
                temperature=0,
                messages=messages,
                stream=True,
            )
            input_tokens += num_tokens_from_messages(messages, model=llm)

            new_output = ""

            for chunk in stream:
                if chunk.choices[0].delta.content is not None:
                    print(chunk.choices[0].delta.content, end="")
                    new_chunk = chunk.choices[0].delta.content
                    new_output += new_chunk

                if STOP_SIGNAL in new_output:
                    break

            output_tokens += num_tokens_from_string(new_output, model=llm)       
            output += new_output
            if not new_output or STOP_SIGNAL in new_output:
                break

            messages.append({"role": "assistant", "content": new_output})
        except Exception as e:
            print("Error:", e)
            # Handle the error here, such as retrying or terminating the loop
    
    output = output.replace("STOPSTOP", "")
    return output, input_tokens, output_tokens


draft_markdown, input_tokens, output_tokens = generate_output(system_prompt, user_prompt, llm)

---
title: Interview Guide for  Product Designer
subtitle:  Tiny
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: January 23, 2024 2:45 PM
--- 

# Interview Guide for Product Designer

## About this Guide

This guide is a tool for interviewers at Tiny to assess candidates for the Product Designer role. It outlines the job description, provides a structured interview agenda, and details the competencies required for the role. The guide also includes questions to probe each competency, situational scenarios, and a case study for role play. It concludes with a candidate Q&A section and an evaluation scorecard. This guide ensures a consistent and thorough evaluation of each candidate's fit for the role and alignment with Tiny's values.

## Job Description

**Job Title: Product Designer**

**About Tiny**

If you've ever written content on the web, chances are you've used a Tiny product. As part of the Tiugo Technologies portfolio of brands, we make TinyMCE 

In [ ]:
# give update on the cost

cost = (input_tokens / 1000 * 0.01) + (output_tokens / 1000 * 0.03)
rounded_cost = round(cost, 2)
print("Total cost is approximately $" + str(rounded_cost) + " USD.")

Total cost is approximately $0.14 USD.


## Produce the Final Outputs

In [ ]:

# Define the file path
file_path = draft_destination

# Save the final_markdown to the file
with open(file_path, "w") as file:
    file.write(draft_markdown)

# Print the file path
print("Draft markdown saved to:", file_path)

Draft markdown saved to: examples/real-estate-litigator/real-estate-litigator-draft.md


In [ ]:
user_prompt = f"""
You are a Talent Acquisition and Development Specialist. Your task is to review a hiring guide that is in draft. 
Please review and improve it as necessary. 

Job description is as follows:
\"\"\"
{job_description}
\"\"\"

Company description is as follows:
\"\"\"
{company_description}
\"\"\"

Draft interview guide is as follows:
\"\"\"
{draft_markdown}
\"\"\"

The template is as follows:
\"\"\"
{document_template}
\"\"\"

"""
output, input_tokens, output_tokens = generate_output(system_prompt, user_prompt, llm)
final_markdown = output.replace("STOPSTOP", "")

# Define the file path
file_path = markdown_destination

# Save the final_markdown to the file
with open(file_path, "w") as file:
    file.write(final_markdown)

# Print the file path
print("Final markdown saved to:", file_path)

---
title: Interview Guide for Real Estate Litigation Attorney
subtitle: Maxwell Fairview
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: January 23, 2024 2:18 PM
--- 

# Interview Guide for Real Estate Litigation Attorney

<!-- The guide is designed to be direct and efficient, reflecting the firm's values of excellence and diversity. It will help the interviewers to assess candidates' technical skills, cultural fit, and alignment with Maxwell Fairview's values. -->

## About this Guide

<!-- This guide is crafted to align with Maxwell Fairview's mission and values, ensuring that the interview process is not only about assessing technical legal skills but also about finding a candidate who embodies the firm's commitment to service, experience, trust, and inclusion. -->

This guide serves as a structured approach for evaluating candidates for the Real Estate Litigation Attorney position at Maxwell Fairview. It is crafted to align with the firm's missio

In [ ]:
# update on the cost
cost = (input_tokens / 1000 * 0.01) + (output_tokens / 1000 * 0.03)
rounded_cost = round(cost, 2)
print("Total cost is approximately $" + str(rounded_cost) + " USD.")

Total cost is approximately $0.18 USD.


## Diff

In [ ]:
import difflib

# Split the markdown strings into lines
draft_lines = draft_markdown.splitlines()
final_lines = final_markdown.splitlines()

# Compute the diff between the two sets of lines
diff = difflib.unified_diff(draft_lines, final_lines)

# Print the diff
for line in diff:
    print(line)


--- 

+++ 

@@ -7,17 +7,17 @@

 
 # Interview Guide for Real Estate Litigation Attorney
 
-<!-- I will create a guide that is straightforward and to the point, focusing on the essential competencies required for the Real Estate Litigation Attorney role at Maxwell Fairview. -->
+<!-- The guide is designed to be direct and efficient, reflecting the firm's values of excellence and diversity. It will help the interviewers to assess candidates' technical skills, cultural fit, and alignment with Maxwell Fairview's values. -->
 
 ## About this Guide
 
-<!-- This section will briefly explain the purpose of the guide, how it should be used, and its alignment with Maxwell Fairview's values. -->
-
-This guide is a tool for hiring teams at Maxwell Fairview to assess candidates for the Real Estate Litigation Attorney position. It outlines the job description, interview agenda, competencies, and evaluation methods. The guide is designed to ensure a fair and effective interview process that aligns wi

# Final Output

In [ ]:
# Convert from markdown to docx
!pandoc -s {markdown_destination} -o {word_destination} --reference-doc="templates/custom-reference.docx" --toc=True --toc-depth=1 --shift-heading-level-by -1

In [ ]:

# display the final_markdown in the notebook
from IPython.display import Markdown
Markdown(final_markdown)

---
title: Interview Guide for Real Estate Litigation Attorney
subtitle: Maxwell Fairview
author: Created by [Deliverables AI](https://github.com/androb/deliverables)
date: January 23, 2024 2:18 PM
--- 

# Interview Guide for Real Estate Litigation Attorney

<!-- The guide is designed to be direct and efficient, reflecting the firm's values of excellence and diversity. It will help the interviewers to assess candidates' technical skills, cultural fit, and alignment with Maxwell Fairview's values. -->

## About this Guide

<!-- This guide is crafted to align with Maxwell Fairview's mission and values, ensuring that the interview process is not only about assessing technical legal skills but also about finding a candidate who embodies the firm's commitment to service, experience, trust, and inclusion. -->

This guide serves as a structured approach for evaluating candidates for the Real Estate Litigation Attorney position at Maxwell Fairview. It is crafted to align with the firm's mission and values, ensuring a comprehensive assessment of both technical legal skills and cultural fit. The guide provides a clear framework for the interview process, from job description to competency evaluation.

The competencies are derived from the job requirements and are infused with Maxwell Fairview's values. They are designed to identify candidates who are not only technically proficient but also demonstrate a commitment to service, trust, and inclusion. This guide will assist interviewers in conducting a fair, consistent, and effective interview.

Interviewers should use this guide to navigate the interview process, ensuring that all relevant areas are covered. It will help in asking pertinent questions, evaluating candidates fairly, and ultimately selecting the best candidate for the role. The guide is a reflection of Maxwell Fairview's dedication to maintaining high standards of legal practice and ethical conduct.

## Job Description

**Real Estate Litigation Attorney**

**Position:** Real Estate Litigation Attorney  
**Location:** San Francisco, CA  
**Practice Area:** Construction & Real Estate  
**Open Date:** Friday, December 15, 2023  
**Job Function:** Legal  

**Job Description:**  
At Maxwell Fairview, our priority is to attract, retain, and promote talented individuals from a wide range of racial, ethnic, social, economic, religious, and personal backgrounds, genders, and sexual orientations. Therefore, we encourage talented individuals with diverse backgrounds and experiences to apply.

More than a third of Maxwell Fairview’s lawyers are litigators. We have decades of experience representing clients at every level of the trial and appellate courts in multiple jurisdictions, in international forums, and mediations, arbitrations, and other methods of alternative dispute resolution. 

A career at Maxwell Fairview is the opportunity to do work that matters. It’s a chance to use your knowledge to shape what’s ahead, to innovate, and to learn at a firm that taps into the power of collaboration and collective thinking. 

In accordance with applicable Federal and State laws, the anticipated annual salary range for this position is as follows:

- San Francisco, CA: $160,000-$175,000

In addition to a standard benefits package, this role may be eligible for additional contingent compensation based on an array of factors, including but not limited to: work performance, geographic location, work experience, education, and qualifications. Because such contingent compensation is not yet calculable or may be zero ($0) in some circumstances, the above-listed salary range is Maxwell Fairview’s good faith estimate of the annual salary it reasonably expects to pay for the position at the time of this posting.

**Job Requirements:**  
Our Construction and Real Estate Litigation practice group is seeking to hire a department attorney to join our San Francisco, CA office. The successful candidate will work with our team on California landlord-tenant matters.  

Qualified candidates will have at least three to six years of commercial and real estate litigation experience. Ability to work in a fast-paced client-centric environment with a varied caseload involving complex issues is essential. California Bar admission is required.

Principals Only.

Maxwell Fairview LLP is an Equal Opportunity / Affirmative Action Employer.
Disability / Female / Gender Identity / Minority / Sexual Orientation / Veteran. 

Pursuant to the San Francisco Fair Chance Ordinance and the Los Angeles Fair Chance Initiative, we will consider for employment qualified applicants with arrest and conviction records. 

To comply with Federal law, Maxwell Fairview participates in E-Verify. All newly-hired employees are verified through this electronic system established by the Department of Homeland Security (DHS) and the Social Security Administration (SSA) to confirm their identity and employment eligibility. Please refer to the Notice of E-Verify Participation and the Right to Work posters on the Maxwell Fairview Careers page for more information.

## Interview Agenda

<!-- The agenda is structured to provide a comprehensive view of the candidate's abilities and to assess their fit with Maxwell Fairview's culture and values. -->

1. Introduction (5 minutes)
2. Overview of Maxwell Fairview and the Real Estate Litigation Attorney role (10 minutes)
3. Discussion of candidate's background and experience (20 minutes)
4. Competency-based questions (30 minutes)
5. Case study or role play exercise (20 minutes)
6. Candidate's questions (10 minutes)
7. Closing remarks and next steps (5 minutes)

## Competency Framework

<!-- The competencies are selected to reflect the specific skills and attributes needed for the Real Estate Litigation Attorney role, as well as the candidate's alignment with Maxwell Fairview's values. -->

The competency framework for the Real Estate Litigation Attorney role at Maxwell Fairview is designed to identify the key attributes required for success in this position. It includes a mix of technical skills, personal characteristics, and alignment with the firm's values. Each competency is critical to the role and reflects the firm's commitment to excellence and diversity.

1. **Legal Expertise**: Knowledge of commercial and real estate litigation, specifically California landlord-tenant law.
2. **Analytical Thinking**: Ability to analyze complex legal issues and develop strategic solutions.
3. **Communication Skills**: Clear and effective communication, both written and oral.
4. **Client-Centric Approach**: Dedication to client service and understanding client needs.
5. **Collaborative Work Ethic**: Ability to work well in a team and contribute to a collaborative environment.
6. **Diversity and Inclusion**: Commitment to diversity, equity, and inclusion in the workplace.

1. **Legal Expertise**: Deep understanding of commercial and real estate litigation, with a focus on California landlord-tenant matters. This competency aligns with Maxwell Fairview's value of Rich Experience.

### Lead Question

**&ldquo;Describe your experience with commercial and real estate litigation, particularly in California landlord-tenant law.&rdquo;** Can you provide specific examples of cases you have worked on? What was your role in these cases?

&nbsp;

&nbsp;

_Look for detailed knowledge of relevant laws and regulations, as well as practical experience in litigation._

### Situational Scenario

**&ldquo;Imagine you are representing a client in a complex landlord-tenant dispute. How would you approach the case from initial consultation to resolution?&rdquo;**

&nbsp;

&nbsp;

_Look for a strategic approach to litigation, including case assessment, development of legal strategy, and execution._

### Diving Deeper

- &ldquo;How do you stay updated on changes in real estate law?&rdquo;
- &ldquo;Can you discuss a time when you had to adapt your legal strategy due to new information or developments in a case?&rdquo;
- &ldquo;How do you balance the need to be thorough with the demands of a fast-paced legal environment?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Limited understanding of relevant laws | Basic understanding with some practical experience | Good knowledge with solid case examples | Strong expertise with significant case experience | Expert level with extensive experience and proven results |

2. **Analytical Thinking**: The ability to dissect complex legal issues and think critically to devise effective strategies. This competency is essential for navigating the intricacies of litigation and aligns with Maxwell Fairview's value of Unparalleled Service.

### Lead Question

**&ldquo;Walk me through your process for analyzing a new legal issue presented by a client.&rdquo;** How do you break down the problem? What steps do you take to research and form your legal opinion?

&nbsp;

&nbsp;

_Look for a methodical approach to problem-solving and the ability to think critically about legal issues._

### Situational Scenario

**&ldquo;You are faced with a case that has multiple potential outcomes. How do you evaluate the risks and benefits of each possible scenario for your client?&rdquo;**

&nbsp;

&nbsp;

_Look for the ability to conduct a thorough risk assessment and communicate potential outcomes to clients._

### Diving Deeper

- &ldquo;Describe a time when you had to analyze a complex legal document. What was challenging about it?&rdquo;
- &ldquo;How do you ensure your analysis is comprehensive and accounts for all relevant factors?&rdquo;
- &ldquo;Can you give an example of a time when your analytical skills led to a positive outcome for a client?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Struggles with complex issues | Can analyze standard issues | Good at analyzing complex issues and forming opinions | Very skilled in complex analysis and strategy development | Exceptional analytical thinker with a track record of innovative solutions |

3. **Communication Skills**: Effective communication is crucial for a litigation attorney. This competency ensures the candidate can articulate complex legal concepts clearly and persuasively, aligning with Maxwell Fairview's value of Unshakeable Trust.

### Lead Question

**&ldquo;How do you ensure that your communication, both in writing and verbally, is clear and effective for clients, colleagues, and the court?&rdquo;** Can you provide an example of a complex legal argument you successfully communicated?

&nbsp;

&nbsp;

_Look for clarity, persuasiveness, and the ability to tailor communication to different audiences._

### Situational Scenario

**&ldquo;You need to explain a complicated legal strategy to a client who is not familiar with legal jargon. How do you approach this?&rdquo;**

&nbsp;

&nbsp;

_Look for the ability to simplify complex information and ensure client understanding._

### Diving Deeper

- &ldquo;Tell me about a time when you had to persuade a skeptical audience.&rdquo;
- &ldquo;How do you handle difficult conversations with clients or opposing counsel?&rdquo;
- &ldquo;What techniques do you use to ensure your written legal arguments are compelling and clear?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Often unclear or confusing | Usually clear but can improve in persuasion | Clear and somewhat persuasive | Very clear and persuasive in most situations | Exceptionally clear and persuasive in all forms of communication |

4. **Client-Centric Approach**: This competency focuses on the attorney's dedication to understanding and meeting client needs, which is a core aspect of Maxwell Fairview's value of Unparalleled Service.

### Lead Question

**&ldquo;Describe how you have tailored your legal services to meet the specific needs of a client.&rdquo;** What steps did you take to understand their needs? How did you ensure their satisfaction?

&nbsp;

&nbsp;

_Look for examples of personalized service and a focus on client satisfaction._

### Situational Scenario

**&ldquo;A client comes to you with a pressing legal issue that requires immediate attention. How do you prioritize their needs while managing your existing caseload?&rdquo;**

&nbsp;

&nbsp;

_Look for the ability to manage time effectively and prioritize client needs._

### Diving Deeper

- &ldquo;How do you build trust with new clients?&rdquo;
- &ldquo;Can you give an example of a time when you went above and beyond for a client?&rdquo;
- &ldquo;How do you measure client satisfaction?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Rarely considers client needs | Sometimes considers client needs | Often meets client needs effectively | Consistently prioritizes and meets client needs | Always exceeds client expectations with exceptional service |

5. **Collaborative Work Ethic**: The ability to work well with others and contribute to a team is essential. This competency reflects Maxwell Fairview's emphasis on collaboration and collective thinking.

### Lead Question

**&ldquo;Tell me about a time when you had to collaborate with a team to achieve a legal objective. How did you contribute?&rdquo;** What challenges did you face, and how did you overcome them?

&nbsp;

&nbsp;

_Look for examples of teamwork and the ability to work collaboratively._

### Situational Scenario

**&ldquo;You are part of a legal team working on a high-stakes case with tight deadlines. How do you ensure effective collaboration and communication among team members?&rdquo;**

&nbsp;

&nbsp;

_Look for strategies to foster teamwork and manage group dynamics._

### Diving Deeper

- &ldquo;How do you handle disagreements within a team?&rdquo;
- &ldquo;Can you share an experience where you had to lead a team? What was your approach?&rdquo;
- &ldquo;What do you think is the key to successful collaboration in a legal setting?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Struggles with teamwork | Can work in a team but prefers solo work | Good team player | Strong collaborator and team contributor | Exceptional leader and team player who enhances group performance |

6. **Diversity and Inclusion**: Maxwell Fairview values diversity and inclusion. This competency ensures the candidate respects and promotes these values in the workplace.

### Lead Question

**&ldquo;How have you contributed to creating an inclusive environment in your previous roles?&rdquo;** Can you share specific actions or initiatives you've been a part of?

&nbsp;

&nbsp;

_Look for a commitment to diversity, equity, and inclusion._

### Situational Scenario

**&ldquo;You notice a lack of diversity in your team or among the clients you serve. What steps would you take to address this issue?&rdquo;**

&nbsp;

&nbsp;

_Look for proactive measures to promote inclusivity._

### Diving Deeper

- &ldquo;How do you ensure that your own biases do not affect your legal work?&rdquo;
- &ldquo;Can you discuss a time when you advocated for diversity and inclusion in the workplace?&rdquo;
- &ldquo;What does an inclusive workplace look like to you, and how do you contribute to it?&rdquo;

&nbsp;

&nbsp;

### Evaluation

| Rating 1 | Rating 2 | Rating 3 | Rating 4 | Rating 5 |
| -------- | -------- | -------- | -------- | -------- |
| Unaware of diversity issues | Recognizes diversity issues but takes no action | Supports diversity initiatives | Actively promotes diversity and inclusion | Leader in creating and sustaining an inclusive environment |

## Case Study / Role Play

<!-- The case study will provide a realistic scenario for candidates to demonstrate their practical skills and problem-solving abilities, as well as their alignment with the firm's values. -->

Candidates will be presented with a case study that simulates a real-world legal scenario relevant to Maxwell Fairview's practice. They will be asked to analyze the case, develop a legal strategy, and present their approach to the interview panel. This exercise will test their legal expertise, analytical thinking, and communication skills.

The evaluation will focus on the candidate's ability to apply their knowledge in a practical setting, their strategic thinking, and their ability to communicate complex legal strategies effectively. It will also assess their ability to work under pressure and their commitment to client-centric service.

## Candidate Q&A

<!-- This section will provide a space for candidates to ask questions about the role, the firm, and anything else they may be curious about. -->

At the end of the interview, candidates will have the opportunity to ask questions. This is an important part of the process, as it allows candidates to demonstrate their interest in the role and Maxwell Fairview. It also gives the interview panel insight into the candidate's priorities and values.

**Q. What are the growth opportunities for a Real Estate Litigation Attorney at Maxwell Fairview?**

A. Maxwell Fairview offers a range of professional development and advancement opportunities, including mentorship, continuing legal education, and leadership roles within the firm.

**Q. How does Maxwell Fairview support work-life balance for its attorneys?**

A. The firm is committed to supporting work-life balance through flexible work arrangements, wellness programs, and a supportive culture.

**Q. Can you tell me more about Maxwell Fairview's commitment to diversity and inclusion?**

A. Maxwell Fairview actively promotes diversity and inclusion through its hiring practices, professional development programs, and community engagement initiatives.

**Q. What types of clients does the Construction and Real Estate Litigation practice group typically represent?**

A. The practice group represents a diverse client base, including property developers, landlords, tenants, and construction companies.

**Q. How does Maxwell Fairview stay ahead of the curve in legal innovation?**

A. The firm invests in technology and encourages a culture of innovation to enhance client services and streamline legal processes.

**Q. What is the firm's approach to alternative dispute resolution?**

A. Maxwell Fairview advocates for alternative dispute resolution methods when appropriate, to achieve efficient and effective outcomes for clients.

## Candidate Evaluation

Complete the following scorecard for the Real Estate Litigation Attorney role at Maxwell Fairview. Refer to the evaluation guides for each competency to determine the candidate's score.

| Competency | Rating (1-5) | Interviewer Comments |
| --- | --- | --- |
| Legal Expertise |   |   |
| Analytical Thinking |   |   |
| Communication Skills |   |   |
| Client-Centric Approach |   |   |
| Collaborative Work Ethic |   |   |
| Diversity and Inclusion |   |   |
| &nbsp; | | |
| **Total** |   |   |

